<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 使用 TAO 工具包训练模型 #
作为回顾，此图说明了典型的模型开发工作流程。首先，准备数据和预训练模型。接着，使用新数据训练模型并评估其性能。在取得令人满意的模型性能后，将其导出。_请注意：本部分内容不包括推理优化，我们将在下个 Notebook 中介绍相关内容。_

<p><img src='images/pre-trained_model_workflow.png' width=1080></p>

## 学习目标 ##
在此 Notebook 中，您将学习如何使用 TAO 工具包训练视频 AI 模型，学习内容包括：
* 采用专用的 TrafficCamNet 模型并训练用于物体检测的自定义 ResNet18 DetectNet_v2 模型
* 创建模型训练、评估和推理配置文件
* 评估模型
* 将模型部署到 DeepStream

**目录** <br>
本 Notebook 涵盖以下部分：
1. [模型训练](#s1)
    * [练习 #1 - 修改数据集配置](#e1) 
    * [练习 #2 - 修改数据增强的配置](#e2) 
    * [练习 #3 - 修改模型配置](#e3)
    * [练习 #4 - 修改边界框栅格化的配置](#e4)
    * [练习 #5 - 修改后处理配置](#e5)
    * [练习 #6 - 修改训练配置](#e6)
    * [代价函数配置](#s1.1)
    * [练习 #7 - 修改评估配置](#e7)
    * [启动模型训练](#s1.2)
2. [评估模型](#s2)
3. [模型推理](#s3)
    * [练习 #8 - 修改推理器配置](#e8)
    * [练习 #9 - 修改边界框处理器配置](#e9)
    * [可视化推理](#s3.1)
4. [模型导出](#s4)
    * [将模型导出为 TensorRT 引擎](#s4.1)
5. [部署到 DeepStream](#s5)

<a name='s1'></a>
## 模型训练 ##
训练配置是通过训练规范文件完成的，这类文件中包括用于训练的数据集、用于验证的数据集、要使用的预训练模型架构、用于调优的超参数等训练选项。DetectNet_v2 实验的 `train` 和 `evaluate` 子任务共用同一个配置文件。可以从头开始创建配置文件，也可以参照 TAO 工具包的[示例应用](https://docs.nvidia.com/tao/tao-toolkit/#cv-applications)中提供的模板加以修改。

训练配置是通过训练规范文件完成的，该文件包括选项，例如用于训练的数据集、用于验证的数据集、要使用的预训练模型架构、要调整的超参数以及其他训练选项。 DetectNet_v2 实验的训练和评估子任务共享相同的配置文件。 可以使用 TAO Toolkit 的示例应用程序中提供的模板从头开始创建或修改配置文件。

<p><img src='images/rewind.png' width=720></p>

_请注意：在使用 NGC 专用模型时，需有正确的**加密密钥**方可加载模型。用户通过通用模型开展训练时，将能够定义自己的导出加密密钥。此操作可保护专有 IP，并用于解密 DeepStream 应用中的 `.etlt` 模型。_

<p><img src='images/encryption_key.png' width=540></p>

训练配置文件包含八个部分：
* `dataset_config`
* `augmentation_config`
* `model_config`
* `bbox_rasterizer_config`
* `postprocessing_config`
* `training_config`
* `cost_function_config`
* `evaluation_config`

<p><img src='images/important.png' width=720></p>
我们将使用模板创建配置文件。具体而言，为便于讨论，我们已将配置文件分解为单独的部分，并会在最后将这些部分组合起来，以供 TAO 工具包使用。

执行以下单元，预览将使用的组合训练/评估配置文件。该文件目前无法使用，因为我们有意作出了一些修改，需要加以更正。

In [ ]:
# DO NOT CHANGE THIS CELL
# Set and create directories for the TAO Toolkit experiment
import os

os.environ['PROJECT_DIR']='/dli/task/tao_project'
os.environ['SOURCE_DATA_DIR']='/dli/task/data'
os.environ['DATA_DIR']='/dli/task/tao_project/data'
os.environ['MODELS_DIR']='/dli/task/tao_project/models'
os.environ['SPEC_FILES_DIR']='/dli/task/spec_files'

In [ ]:
# DO NOT CHANGE THIS CELL
# Combining configuration components in separate files and writing into one
!cat $SPEC_FILES_DIR/dataset_config.txt \
     $SPEC_FILES_DIR/augmentation_config.txt \
     $SPEC_FILES_DIR/model_config.txt \
     $SPEC_FILES_DIR/bbox_rasterizer_config.txt \
     $SPEC_FILES_DIR/postprocessing_config.txt \
     $SPEC_FILES_DIR/training_config.txt \
     $SPEC_FILES_DIR/cost_function_config.txt \
     $SPEC_FILES_DIR/evaluation_config.txt \
     > $SPEC_FILES_DIR/combined_training_config.txt
!cat $SPEC_FILES_DIR/combined_training_config.txt

<a name='e1'></a>
#### 练习 #1 - 修改数据集的配置 ####
数据加载器定义用于训练的数据的路径，以及数据集中类的类映射。我们之前为训练数据集生成了 TFRecord。要使用新生成的 TFRecord，请更新规格文件中的 `dataset_config` 参数，以引用正确目录。另一个要考虑的参数是 `validation_fold`，我们可以用 `0` 来表明 _随机拆分数据_ 。如要按序列拆分，我们可以使用从数据集转换工具生成的任何文件。
* `data_sources (dict)`：捕获要用于训练的 TFRecord 的路径。
    * `tfrecords_path (str)`：训练 TFRecord 根/TFRecords_name* 的路径，即 **/data/tfrecords/kitti_trainval/*** 。
    * `image_directory_path (str)`：从中生成 TFRecord 的训练数据源的路径。
* `image_extension (str)`：图像扩展名，无需 `.` 即可使用。
* `target_class_mapping (dict)`：此参数会将 TFRecord 中的类名映射到要训练的目标类。该字段支持将相似的类对象结组到同一个类下。
    * `key (str)`：TFRecord 文件中的类名称值。该值必须与数据集转换器日志中显示值的相同。
    * `value (str)`：对应于网络的预期学习值。
* `validation_fold (int)`：如果是 N 折的 TFRecord，您可以定义用于验证的折叠索引值。对于*随机拆分的*分区，请将验证折叠索引值强制设置为 0，因为 TFRecord 仅为 2 折。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并**保存更改**，完成对训练配置文件 `dataset_config`[（此处）](spec_files/dataset_config.txt)部分的修改。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/dataset_config.txt

In [ ]:
# dataset_config: {
#   data_sources: {
#     tfrecords_path: "/dli/task/tao_project/data/tfrecords/kitti_trainval/*"
#     image_directory_path: "/dli/task/tao_project/data/training"
#   }
#   image_extension: "png"
#   target_class_mapping: {
#       key: "car"
#       value: "car"
#   }
#   validation_fold: 0
# }

点击 ... 以显示**答案**。

<a name='#e2'></a>
#### 练习 #2 - 修改数据增强的配置 ####
在使用自己的数据集训练和微调模型时，可以在训练时扩充数据集以引入数据的变化。这即是我们所说的**在线增强**。此操作对训练非常有用，因为数据的变化可以提高模型的整体质量，并防止[过度拟合](https://en.wikipedia.org/wiki/Overfitting)。训练深度神经网络需要大量的标注数据，这一过程可能需要手动操作，且成本高昂。此外，我们也难以估计网络可能经历的所有极端情况。TAO 工具包具备*空间增强*（旋转、调整大小、转换、剪切和翻转）、*色彩空间增强*（色调旋转、亮度偏移和对比度偏移）以及*图像模糊*功能，可用于创建合成的数据变体。

如要了解其中的部分值，则需要查看[模型卡](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/trafficcamnet)。

<p><img src='images/model_card_tao.png' width=720></p>

* `preprocessing (dict)`：配置输入图像和真值标签预处理模组。
    * `output_image_width (int)`：与网络输入的宽度相同。
    * `output_image_height (int)`：与网络输入的高度相同。
    * `output_image_channel (int)`：与网络输入 _(1, 3)_ 的通道深度相同。
    * `min_bbox_width (float)`：意在用于训练的对象标签最小宽度。
    * `min_bbox_height (float)`：意在用于训练的对象标签最小宽度。
* `spatial_augmentation (dict)`：支持空间增强，例如翻转、缩放和转换。
    * `hflip_probability (float)`：水平翻转输入图像的概率 _(0.0 - 1.0)_ 。
    * `vflip_probability (float)`：垂直翻转输入图像的概率 _(0.0 - 1.0)_ 。
    * `zoom_min (float)`：输入图像的最小缩放比例 _(> 0.0)_ 。
    * `zoom_max (float)`：输入图像的最大缩放比例 _(> 0.0)_ 。
    * `translate_max_x (int)`：在 x 轴中添加的最大转换值 _(0.0 - output_image_width)_ 。
    * `translate_max_y (int)`：在 y 轴中添加的最大转换值 _(0.0 - output_image_height)_ 。
    * `rotate_rad_max (float)`：应用于图像和训练标签的旋转角度 _(> 0.0)_ 。
* `color_augmentation (dict)`：配置色彩空间转换。
    * `color_shift_stddev (float)`：色移的标准偏差值 _(0.0 - 1.0)_ 。
    * `hue_rotation_max (float)`：色调旋转矩阵的最大旋转角度 _(0.0 - 360.0)_ 。
    * `saturation_shift_max (float)`：更改饱和度的最大偏移值 _(0.0 - 1.0)_ 。
    * `contrast_scale_max (float)`：围绕给定中心旋转的对比度斜率 _(0.0 - 1.0)_ 。
    * `contrast_center (float)`：对比度围绕其旋转的中心 _（设置为 0.5）_ 。

**注意**：如果预处理块的输出图像高度和输出图像宽度与生成 TFRecord 时提到的输入图像的高度和宽度不符，系统会随机填充或裁剪图像以使之符合输入分辨率。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并**保存更改**，完成对训练配置文件 `augmentation_config`[（此处）](spec_files/augmentation_config.txt)部分的修改。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/augmentation_config.txt

In [ ]:
# augmentation_config: {
#   preprocessing: {
#     output_image_width: 960
#     output_image_height: 544
#     output_image_channel: 3
#     min_bbox_width: 1.0
#     min_bbox_height: 1.0
#   }
#   spatial_augmentation: {
#     hflip_probability: 0.5
#     vflip_probability: 0.5
#     zoom_min: 1.0
#     zoom_max: 1.0
#     translate_max_x: 8.0
#     translate_max_y: 8.0
#   }
#   color_augmentation: {
#     color_shift_stddev: 0.0
#     hue_rotation_max: 25.0
#     saturation_shift_max: 0.2
#     contrast_scale_max: 0.1
#     contrast_center: 0.5
#   }
# }

点击 ... 以显示**解决方案**。

<a name='e3'></a>
#### 练习 #3 - 修改模型的配置 ####
可以使用规格文件中的 `model_config` 选项配置核心物体检测模型。
* `arch (str)`：主干特征提取器的体系架构 _（默认值为"resnet"）_ 。
* `pretrained_model_file (str)`：预训练 TAO 模型文件的路径。
* `num_layers (int)`：可扩展模板特征提取器的深度。
* `use_pooling (bool)`：在缩减取样时选择使用“跨步卷积”或“最大池化”。我们建议将此项设置为 `false` 并使用跨步卷积。
* `objective_set (dict)`：网络训练目标。对于物体检测网络，将其设置为学习 `cov` 和 `bbox`。这些设置不应改变。
    * `bbox`
        * `scale`: `35.0`
        * `offset`: `0.5`
    * `cov`
* `dropout_rate (float)`：丢弃概率 _(0.0 - 1.0)_ 。
* `load_graph (bool)`：用以确定是否从预训练模型加载图形或是否仅加载权重的标记。对于已剪枝的模型，请将此参数设置为 `true`，因为需要同时导入模型图形和权重。
* `freeze_blocks (float)`：定义或需从实例化特征提取器模板中冻结的块。在神经网络上下文中冻结层是为了控制权重是否更新。某个层（如特征提取器中的层）遭到冻结时，即表示无法进一步修改权重。此技术用于缩短训练的计算时间。如需了解有关冻结层的更多信息，请参阅 [TensorFlow 的迁移学习和微调指南](https://www.tensorflow.org/guide/keras/transfer_learning#freezing_layers_understanding_the_trainable_attribute)。
* `use_batch_norm (bool)`：用以确定是否使用了批量归一化的标记 _（默认=false）_ 。批量归一化是一种用于训练深度较高的神经网络（如我们正在使用的此类网络）的技术，能够对每个小批量的输入层执行标准化处理。该技术旨在应对**内部协变量偏移**，当每个小批量的权重更新后，网络中各深度层的输入分布发生变化时，就会出现这种情况。批量归一化可以稳定学习过程，并显著减少训练深度网络所需的训练次数。您可以参阅 [TensorFlow 的 API 文档](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization)，了解更多有关分批标准化的信息。
* `freeze_bn (bool)`：确定是否在训练期间冻结模组中的批量归一化层。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并**保存更改**，完成对训练配置文件 `model_config`[（此处）](spec_files/model_config.txt)部分的修改。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/model_config.txt

In [ ]:
# model_config: {
#   arch: "resnet"
#   pretrained_model_file: "/dli/task/tao_project/models/trafficcamnet_vunpruned_v1.0/resnet18_trafficcamnet.tlt"
#   freeze_blocks: 0
#   freeze_blocks: 1
#   num_layers: 18
#   use_pooling: false
#   use_batch_norm: true
#   dropout_rate: 0.0
#   objective_set: {
#     cov: {}
#     bbox: {
#       scale: 35.0
#       offset: 0.5
#     }
#   }
# }

点击 ... 以显示**解决方案**。

<a name='e4'></a>
#### 练习 #4 - 修改边界框格栅化的配置 ####
DetectNet_v2 生成 2 个张量：`cov` 和 `bbox`。系统会将图像分成 16x16 网格单元。`cov` (_coverage_) 张量定义了单个物体所覆盖的网格单元数量。`bbox` 张量定义了物体左上角和右下角相对于网格单元的标准化图像坐标。
* `deadzone radius (float)`：物体椭圆区域周围被视为静止状态的区域。这在物体重叠的情况下非常有用，可避免混淆前景物体和背景物体 _(0.0 - 1.0)_ 。
* `target_class_config (dict)`：定义给定类物体的覆盖区域，并为每个类重复。
    * `cov_center_x (float)`：物体中心的 x 坐标 _(0.0 - 1.0)_ 。
    * `cov_center_y (float)`：物体中心的 y 坐标 _(0.0 - 1.0)_ 。
    * `cov_radius_x (float)`：覆盖区域椭圆的 X 半径 _(0.0 - 1.0)_ 。
    * `cov_radius_y (float)`：覆盖区域椭圆的 y 半径 _(0.0 - 1.0)_ 。
    * `bbox_min_radius (float)`：要为框绘制的覆盖区域的最小半径 _(0.0 - 1.0)_ 。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并**保存更改**，完成对训练配置文件 `bbox_rasterizer_config`[（此处）](spec_files/bbox_rasterizer_config.txt)部分的修改。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/bbox_rasterizer_config.txt

In [ ]:
# bbox_rasterizer_config: {
#   target_class_config: {
#     key: "car"
#     value: {
#       cov_center_x: 0.5
#       cov_center_y: 0.5
#       cov_radius_x: 0.4
#       cov_radius_y: 0.4
#       bbox_min_radius: 1.0
#     }
#   }
#   deadzone_radius: 0.4
# }

点击 ... 以显示**解决方案**。

<a name='e5'></a>
#### 练习 #5 - 修改后处理配置 ####
后处理器模组在原始检测输出中生成可渲染的边界框。此过程包括：
1. 使用覆盖区域张量中的置信度值为物体进行阈值化处理，从而过滤出有效的检测结果。
2. 使用 DBSCAN 聚类分析原始过滤预测结果，生成最终渲染的边界框。
3. 根据聚类到同一个分类的候选框获得最终的置信度阈值，并根据这个阈值过滤掉置信度较低的聚类。
* `target_class_config (dict)`：对于正在训练的每个类，`postprocessing_config` 具有 `target_class_config` 元素，用以定义该类的聚类参数。
    * `key (str)`：分类名称。
    * `value (dict)`：配置后处理器模组的聚类分析 config proto 参数。
        * `cluster_config (dict)`: 
            * `coverage_threshold (float)`：将覆盖范围张量的输出视为聚类有效候选框所需到达到的最小阈值 _(0.0 - 1.0)_ 。
            * `dbscan_eps (float)`：两个样本之间的最大距离，将其中一个样本视为在另一个样本的附近。dbscan_eps 越大，分组到一起的框越多 _(0.0 - 1.0)_ 。
            * `dbscan_min_samples (float)`：一个点在邻域中被视为核心点所需达到的总权重 _(0.0 - 1.0)_ 。
            * `minimum_bounding_box_height (int)`：视为有效检测后聚类的最小高度（以像素为单位）_（0 - 输入图像高度）_ 。
            * `clustering_algorithm (enum)`：定义后处理算法（DBSCAN，即基于密度的聚类算法；NMS，即非极大值抑制；或混合方法），以将原始检测结果聚类到最终的边界框渲染（默认 = DBSCAN）。使用混合模式时，请确保定义 DBSCAN 和 NMS 配置参数。
            * `DBSCAN` -`dbscan_confidence_threshold (float)`：用于从 DBSCAN 过滤出聚类边界框输出的置信阈值 _(> 0.0，默认=0.1)_ 。
            * `NMS` - `nms_iou_threshold (float)`：交并比阈值，以过滤出原始检测中的冗余框，形成最终的聚类输出 _(0.0 - 1.0，默认=0.2)_ 。
            * `NMS` -`nms_confidence_threshold (float)`：用于从 NMS 过滤出聚类边界框输出的置信阈值 _(0.0 - 1.0，默认=0)_ 。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并**保存更改**，完成对训练配置文件 `postprocessing_config`[（此处）](spec_files/postprocessing_config.txt)部分的修改。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/postprocessing_config.txt

In [ ]:
# postprocessing_config: {
#   target_class_config: {
#     key: "car"
#     value: {
#       clustering_config: {
#         coverage_threshold: 0.005
#         dbscan_eps: 0.15
#         dbscan_min_samples: 0.05
#         minimum_bounding_box_height: 20
#       }
#     }
#   }
# }

点击 ... 以显示**解决方案**。

<a name='e6'></a>
#### 练习 #6 - 修改训练配置 ####
`training_config` 介绍训练和学习流程。
* `batch_size_per_gpu (int)`：每个 GPU 每批的图像数量 _(> 1)_ 。
* `num_epochs (int)`：运行实验的总迭代次数。
* `learning_rate (dict)`：定义学习率时间表。目前 DetectNet_v2 只支持：
    * `soft_start_annealing_schedule (dict)`，使用以下方式进行配置：
        * `soft_start (float)`：将学习率从最低学习率提升至最高学习率的时间 _(0.0 - 1.0)_ 。
        * `annealing (float`：将学习率从最高学习率冷却至最低学习率的时间 _(0.0 - 1.0)_ 。
        * `minimum learning rate (float)`：学习率时间表中的最低学习率 _(0.0 - 1.0)_ 。
        * `maximum learning rate (float)`：学习率时间表中的最高学习率 _(0.0 - 1.0)_ 。
* `regularizer (dict)`：训练期间要使用的正则化的类型和权重。**建议从低正则化权重开始并逐渐对其进行微调，以缩小训练和验证准确度之间的差距。此外，根据我们的实验结果，L1 似乎为我们提供了更好的剪枝比。**
    * `type (enum)`：支持的类型为 _（NO_REG、L1、L2）_ 。
    * `weight (float)`：正则化器的权重。
* `optimizer (dict)`：用于训练的优化器。
    * `adam (dict)`
        * `epsilon (float)`：避免在实现中被零除的小数字。
        * `beta1 (float)`
        * `beta2 (float)`
* `cost_scaling (dict)`：在训练期间启用成本扩展。**目前请为 DetectNet_v2 训练通道保持此参数不变**。
    * `enabled`: false。
    * `initial_exponent`: 20.0。
    * `increment`: 0.005。
    * `derement`: 1.0。
* `checkpoint_interval (int)`：`train` 保存中间模型的时间间隔（迭代次数） _(0 - num_epochs)_ 。
* `enable_qat (bool)`：使用量化感知训练 (QAT) 启用模型训练。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并保存更改，完成对训练配置文件 `training_config`[（此处）](spec_files/training_config.txt)部分的修改。先选择较低的 `num_epochs` 值（推荐使用 10）执行迭代，因为迭代次数越多，训练时间越长。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/training_config.txt

In [ ]:
# training_config: {
#   batch_size_per_gpu: 16
#   num_epochs: 10
#   learning_rate: {
#     soft_start_annealing_schedule: {
#       min_learning_rate: 5e-6
#       max_learning_rate: 5e-4
#       soft_start: 0.1
#       annealing: 0.7
#     }
#   }
#   regularizer: {
#     type: L1
#     weight: 3e-9
#   }
#   optimizer: {
#     adam: {
#       epsilon: 1e-08
#       beta1: 0.9
#       beta2: 0.999
#     }
#   }
#   cost_scaling: {
#     enabled: false
#     initial_exponent: 20.0
#     increment: 0.005
#     decrement: 1.0
#   }
#   checkpoint_interval: 5
# }

点击 ... 以显示**解决方案**。

<a name='s1.1'></a>
### 代价函数的配置 ###
代价函数描述每个类的训练方式。为获得最佳性能，除确保每个目标类都有一个条目外，我们不需要更改规格文件中的参数。此处的其他参数应保持不变。以下是示例配置文件：

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/cost_function_config.txt

<a name='e7'></a>
#### 练习 #7 - 修改评估配置 ####
可以使用 `evaluation_config` 参数配置评估器。
* `average_precision_mode (enum)`：计算平均精度的模型 _（示例或集成）_ 。
* `validation_period_during_training (int)`：训练期间执行评估的时间间隔 _（1 - 迭代总次数）_ 。
* `first_validation_epoch (int)`：开始运行验证的首轮迭代 _（1 - 迭代总次数）_ 。
* `minimum_detection_ground_truth_overlap (dict)`：执行聚类分析后，真值与预测框之间的最小交并比，用以调用有效检测。
    * `key (str)`：类名称。
    * `value (float)`：交并比值。
* `evaluation_box_config (dict)`：将最小框和最大框维度配置为有效真值和预测值，以执行平均精度计算。
    * `minimum_height (float)`：有效真值和预测边界框中的最低高度（以像素为单位）_（0.0 - 模型图像高度）_ 。
    * `minimum_width (float)`：有效真值和预测边界框中的最低宽度（以像素为单位）_（0.0 - 模型图像宽度）_ 。
    * `maximum_height (float)`：有效真值和预测边界框中的最高高度（以像素为单位）_（minimum_height - 模型图像高度）_ 。
    * `maximum_width (float)`：有效真值和预测边界框中的最高宽度（以像素为单位）_（minimum_width - 模型图像宽度）_ 。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并**保存更改**，完成对训练配置文件 `evaluation_config`[（此处）](spec_files/evaluation_config.txt)部分的修改。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/evaluation_config.txt

In [ ]:
# evaluation_config: {
#   average_precision_mode: INTEGRATE
#   validation_period_during_training: 5
#   first_validation_epoch: 1
#   minimum_detection_ground_truth_overlap: {
#     key: "car"
#     value: 0.7
#   }
#   evaluation_box_config {
#     key: "car"
#     value: {
#       minimum_height: 4
#       maximum_height: 9999
#       minimum_width: 4
#       maximum_width: 9999
#     }
#   }
# }

点击 ... 以显示**解决方案**。

<a name='s1.2'></a>
### 启动模型训练 ###
使用 `train` 子任务时，`-e` 参数表示规格文件的路径，`-r` 参数表示结果目录，`-k` 表示*加载*预训练权重的密钥，`-n` 表示已保存的最后一步模型的名称。

**注意**：训练可能需要数小时才能完成。Detectnet_v2 支持从检查点重新启动，以防训练作业过早终止。只需重新运行**同**一命令行，即可从距离最近的检查点处恢复训练。

_可以使用 `--gpus` 参数为具有硬件的训练启用多 GPU 支持。在使用多个 GPU 运行训练时，我们需要修改 `batch_size_per_gpu` 和 `learning_rate`，以获得与 1 个 GPU 训练类似的 mAP。在大多数情况下，将批量大小缩小 NUM_GPU 倍或将学习率扩大 NUM_GPU 倍不失为一个不错的方法。_

In [ ]:
# DO NOT CHANGE THIS CELL
# Combining configuration components in separate files and writing into one
!cat $SPEC_FILES_DIR/dataset_config.txt \
     $SPEC_FILES_DIR/augmentation_config.txt \
     $SPEC_FILES_DIR/model_config.txt \
     $SPEC_FILES_DIR/bbox_rasterizer_config.txt \
     $SPEC_FILES_DIR/postprocessing_config.txt \
     $SPEC_FILES_DIR/training_config.txt \
     $SPEC_FILES_DIR/cost_function_config.txt \
     $SPEC_FILES_DIR/evaluation_config.txt \
     > $SPEC_FILES_DIR/combined_training_config.txt
!cat $SPEC_FILES_DIR/combined_training_config.txt

In [ ]:
# DO NOT CHANGE THIS CELL
# View train usage
!detectnet_v2 train --help

In [ ]:
# DO NOT CHANGE THIS CELL
# Initiate the training process
!detectnet_v2 train -e $SPEC_FILES_DIR/combined_training_config.txt \
                    -r $MODELS_DIR/resnet18_detector \
                    -k tlt_encode \
                    -n resnet18_detector

<a name='s2'></a>
## 评估模型 ##
我们会在训练结束时和按特定时间间隔评估模型性能，评估可使用 `evaluation_config` 进行配置。用户可通过评估配置选择用于评估的数据集以及评估指标。我们还可以使用 `evaluate` 子任务来评估模型。该 `evaluate` 子任务在训练期间使用的相同验证集上运行评估，但可以进行更新，以在 `dataset_config` 中包含测试数据集。我们还可以通过编辑规格文件以指向目标模型，进而评估早期模型。使用 `evluate` 子任务时，`-e` 参数表示规格文件的路径，`-m` 参数表示模型的路径，`-k` 参数表示_加载_模型的密钥。

In [ ]:
# DO NOT CHANGE THIS CELL
# View saved weights
print('Model for Each Epoch:')
print('---------------------')

!ls -lh $MODELS_DIR/resnet18_detector/weights

In [ ]:
# DO NOT CHANGE THIS CELL
# View evaluate usage
!detectnet_v2 evaluate --help

In [ ]:
# DO NOT CHANGE THIS CELL
# Evaluate the model using the same validation set as training
!detectnet_v2 evaluate -e $SPEC_FILES_DIR/combined_training_config.txt\
                       -m $MODELS_DIR/resnet18_detector/weights/resnet18_detector.tlt \
                       -k tlt_encode

**观察**：<br>
请记下此模型的 mAP，我们在相当短的时间内训练了该模型。

<a name='s3'></a>
## 模型推理 ##
`infer` 子任务可用于对单张图像或图像目录上的边界框进行可视化处理。这是一项可选操作，但我们强烈建议您在部署模型之前执行此操作。`infer` 子任务会在 `output_path/images_annotated` 目录中生成边界框渲染图像，并在 `output_path/labels` 目录中以 KITTIE 格式生成边界框标签。推理可能需要花费一些时间，具体时长视验证数据集大小而定。

推理规格文件用于选择运行推理的选项。这份文件由两个模块组成：
* inferencer_config
* bbox_handler_config

<a name='e8'></a>
#### 练习 #8 - 修改推理器的配置 ####
推理器可实例化模型对象和预处理管道。
* `inferencer_config (dict)`: 
    * `target_classes (str)`：模型应输出的目标类的名称。对于多类模型，此参数会重复。顺序必须与训练配置文件 cost_function_config 中的类相同。
    * `batch_size (int)`：每个推理批次的图像数。
    * `image_height (int)`：模型将推理的图像高度（以像素为单位）。
    * `image_width (int)`：模型将推理的图像宽度（以像素为单位）。
    * `image_channel (int)`：每张图像的通道数。
    * `tlt_config (dict)`：用以实例化模型对象的 Proto 配置。
        * `model (str)`：`.tlt` 模型文件的路径。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并**保存更改**，完成对推理配置文件 `inferencer_config`[（此处）](spec_files/inferencer_config.txt)部分的修改。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/inferencer_config.txt

In [ ]:
# inferencer_config: {
#   target_classes: "car"
#   image_width: 882
#   image_height: 692
#   image_channels: 3
#   batch_size: 16
#   tlt_config: {
#     model: "/dli/task/tao_project/models/resnet18_detector/weights/resnet18_detector.tlt"
#   }
# }

点击 ... 以显示**解决方案**。

<a name='e9'></a>
#### 练习 #9 - 修改边界框处理器的配置 ####
边界框处理器负责处理后处理、边界框渲染，以及序列化到 KITTI 格式的输出标签。执行的处理步骤如下：
1. 对原始输出进行阈值化，以定义每个类可能存在检测的网格单元。
2. 根据推理器的原始坐标重建图像空间坐标。
3. 聚类分析原始阈值预测。
4. 按类别过滤各类经聚类分析的预测。
5. 以输入维度渲染图像上的最终边界框，并将其序列化为 KITTI 格式的元数据。
* `bbox_handler_config (dict)`: 
    * `kitti_dump (bool)`：启用以 KITTI 格式保存每张图像最终输出预测的标记。
    * `disable_overlay (bool)`：禁用每个图像的边界框渲染的标记。
    * `overlay_linewidth (int)`：边界框边界的厚度（以像素为单位）。
    * `classwise_bbox_handler_config (dict)`：包含用于配置聚类算法和边界框渲染器参数的 Proto 对象。需为每个类重复这一操作。
        * `key (str)`：分类名称。
        * `value (dict)`：配置后处理器模组的聚类分析 config proto 参数。
            * `confidence_model (str)`：用以计算聚类边界框最终置信度的算法 _（"aggregate_cov" 或 "mean_cov"）_ 。
            * `bbox_color (dict)`：每个框的 RGB 通道色彩强度。
                * `R (int)`: _(0 - 255)_ 。
                * `G (int)`: _(0 - 255)_ 。
                * `B (int)`: _(0 - 255)_ 。
            * `cluster_config (dict)`:  
                * `coverage_threshold (float)`：将覆盖范围张量输出视为聚类有效候选框所需到达到的最小阈值 _(0.0 - 1.0)_ 。
                * `dbscan_eps (float)`：两个样本之间的最大距离，将其中一个样本视为在另一个样本的附近。dbscan_eps 越大，分组到一起的框越多 _(0.0 - 1.0)_ 。
                * `dbscan_min_samples (float)`：一个点在邻域中被视为核心点所需达到的总权重 *(0.0 - 1.0)* 。
                * `minimum_bounding_box_height (int)`：视为有效检测后聚类的最小高度（以像素为单位）_（0 - 输入图像高度）_ 。
                * `clustering_algorithm (enum)`：定义后处理算法（DBSCAN，即基于密度的聚类算法；NMS，即非极大值抑制；或混合方法），以将原始检测结果集群到最终的边界框渲染（默认 = DBSCAN）。使用混合模式时，请确保定义 DBSCAN 和 NMS 配置参数。
                * `DBSCAN` -`dbscan_confidence_threshold (float)`：用于从 DBSCAN 过滤出聚类边界框输出的置信阈值 _（> 0.0，默认=0.1）_ 。
                * `NMS` - `nms_iou_threshold (float)`：交并比阈值，以过滤出原始检测中的冗余框，形成最终聚类输出 _（（(0.0 - 1.0)，默认=0.2）_ 。
                * `NMS` -`nms_confidence_threshold (float)`：用于从 NMS 过滤出聚类边界框输出的置信阈值 _（0.0 - 1.0，默认=0）_ 。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受值并**保存更改**，完成对推理配置文件 `bbox_handler_config`[（此处）](spec_files/bbox_handler_config.txt)部分的修改。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/bbox_handler_config.txt

In [ ]:
# bbox_handler_config: {
#   kitti_dump: false
#   disable_overlay: false
#   overlay_linewidth: 2
#   classwise_bbox_handler_config: {
#     key:"car"
#     value: {
#       confidence_model: "aggregate_cov"
#       bbox_color: {
#         R: 0
#         G: 255
#         B: 0
#       }
#       clustering_config: {
#         clustering_algorithm: DBSCAN
#         coverage_threshold: 0.005
#         dbscan_eps: 0.15
#         dbscan_min_samples: 0.05
#         minimum_bounding_box_height: 20        
#       }
#     }
#   }
# }

点击 ... 以显示**解决方案**。

In [ ]:
# DO NOT CHANGE THIS CELL
# Combining configuration components in separate files and writing into one
!cat $SPEC_FILES_DIR/inferencer_config.txt \
     $SPEC_FILES_DIR/bbox_handler_config.txt \
     > $SPEC_FILES_DIR/combined_inference_config.txt
!cat $SPEC_FILES_DIR/combined_inference_config.txt

使用 `inference` 子任务时，`-e` 参数表示推理规格文件的路径，`i` 参数表示输入图像目录的路径，`-o` 参数表示输出图像目录的路径，`-k` 参数表示*加载*模型的密钥。如要对单张图像运行推理，只需将 `inference` 子任务 `-i` 参数的路径替换为图像的路径即可。

In [ ]:
# DO NOT CHANGE THIS CELL
# View inference usage
!detectnet_v2 inference --help

In [ ]:
# DO NOT CHANGE THIS CELL
# Perform inference on the validation set
!detectnet_v2 inference -e $SPEC_FILES_DIR/combined_inference_config.txt \
                        -o $PROJECT_DIR/tao_infer_testing \
                        -i $DATA_DIR/training/images \
                        -k tlt_encode

<a name='s3.1'></a>
### 可视化推理 ###
我们可以编写一个快速函数，帮助我们对随机推理进行抽样。

执行以下单元格以可视化推理。

In [ ]:
# DO NOT CHANGE THIS CELL
# Simple grid visualizer
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
import random

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['PROJECT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path)]
    for idx, img_path in enumerate(random.sample(a, num_images)):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img)

In [ ]:
# DO NOT CHANGE THIS CELL
# Visualizing the random 12 images
OUTPUT_PATH = 'tao_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

<a name='s4'></a>
## 模型导出 ##
可结合使用 `.tlt` 模型与 TAO 工具包以进行推理，但 DeepStream 不支持直接使用。相反，TAO 工具包支持使用 `export` 子任务导出并准备训练模型，以部署到 DeepStream。

我们有两种方式可以在 DeepStream 上部署由 TAO 工具包训练的模型：
1. 像以前一样，使用 `export` 并将其集成到 DeepStream 应用中，以生成 `.etlt` 模型或加密的 TAO 文件。DeepStream 将生成 TensorRT 引擎文件，然后运行推理。
2. 生成特定于设备优化的 TensorRT 引擎，这可以是 `.trt` 或 `.engine` 文件，并将其集成到 DeepStream 中。

<p><img src='images/important.png' width=720></p>

_TensorRT 引擎使用针对特定的机器进行的优化，这个优化是每个硬件配置独有的，因此应针对每个推理环境生成。通常，模型训练环境将拥有比部署环境更多的计算资源。除非部署硬件与训练 GPU 相同，否则使用 TAO 工具包生成的 TensorRT 引擎文件无法用于部署。另一方面，该 `.etlt` 文件可广泛用于训练和部署硬件。_

使用 `export` 子任务时，`-m` 参数表示要导出的 `.tlt` 模型文件的路径，`-o` 参数表示已导出 `.etlt` 模型的保存路径，`-k` 参数表示**加密密钥**，`-e` 参数表示实验规格文件。此外，TAO 工具包支持对模板配置文件进行序列化，以便 DeepStream 的 `Gst-nvinfer` 元素使用此模型。此配置文件包含解析 etlt 模型文件所需的网络特定预处理参数和网络图参数 — 非常方便参考。该文件还会生成一个标签文件 `labels.txt`，其中包含按照生成输出的顺序训练的模型类名称。要生成 DeepStream 模板文件，只需使用 `--gen_ds_config` 选项运行 `export` 子任务即可。<br> 执行以下单元以导出经过训练的模型。

In [ ]:
# DO NOT CHANGE THIS CELL
# View export usage
!detectnet_v2 export --help

In [ ]:
# DO NOT CHANGE THIS CELL
# Removing a pre-existing copy if there has been any.
!rm -rf $MODELS_DIR/resnet18_detector_unpruned
!mkdir -p $MODELS_DIR/resnet18_detector_unpruned

In [ ]:
# DO NOT CHANGE THIS CELL
# Exporting .tlt model
!detectnet_v2 export -m $MODELS_DIR/resnet18_detector/weights/resnet18_detector.tlt \
                     -o $MODELS_DIR/resnet18_detector_unpruned/resnet18_detector.etlt \
                     -k tlt_encode \
                     -e $SPEC_FILES_DIR/combined_training_config.txt \
                     --gen_ds_config

<a name='s5'></a>
## 部署到 DeepStream ##
经训练的模型已准备好部署到 DeepStream 工作流中。我们已有一个脚本，该脚本将根据以下 3 个参数启动 DeepStream 工作流：<br>
`python sample_apps/app_03.py <input video file name> <path to nvinfer configuration file> <output video file name>` <br>
它具有以下架构，与我们在本课程第一部分创建的架构非常相似。

<p><img src='images/deepstream_pipeline.png' width=1080></p>

执行以下单元，通过这个简单的 DeepStream 工作流传输两次视频，一次使用专用的 TrafficCamNet 模型，另一次使用自定义模型。之后，比较结果。<br>
**注意**：如果时间允许，请在课程结束时随意修改一些训练参数，以改进模型。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the nvinfer config file using the TrafficCamNet as is
!cat $SPEC_FILES_DIR/pgie_config_trafficcamnet.txt

In [ ]:
# DO NOT CHANGE THIS CELL
# Run the DeepStream Pipeline with the Purpose-built Model as is
%run sample_apps/app_03.py data/sample_30.h264 spec_files/pgie_config_trafficcamnet.txt output.mp4

# Convert the video to a format that is compatible with Jupyter Lab
!ffmpeg -i output.mp4 output_conv.mp4 -y -loglevel quiet

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the nvinfer config file using the custom model
!cat $SPEC_FILES_DIR/pgie_config_resnet18_detector_unpruned.txt

In [ ]:
# DO NOT CHANGE THIS CELL
# Run the DeepStream Pipeline with the Custom Model
%run sample_apps/app_03.py /dli/task/data/sample_30.h264 spec_files/pgie_config_resnet18_detector_unpruned.txt output_resnet18_detector_unpruned.mp4

# Convert the video to a format that is compatible with Jupyter Lab
!ffmpeg -i output_resnet18_detector_unpruned.mp4 output_resnet18_detector_unpruned_conv.mp4 -y -loglevel quiet

In [ ]:
# DO NOT CHANGE THIS CELL
# Compare results
from IPython.display import HTML

HTML("""
<div>
    <video alt="input" width='49%' autoplay>
        <source src="output_conv.mp4" type="video/mp4">
    </video>
    <video alt="output" width='49%' autoplay>
        <source src="output_resnet18_detector_unpruned_conv.mp4" type="video/mp4">
    </video>
</div>
""")

**您做的非常好**！准备就绪后，我们开始学习[下一个 Notebook](./04_optimizing_a_video_AI_application.ipynb)。

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>